<a href="https://colab.research.google.com/github/LLT9798/Twitter-Demography-Research-Project/blob/master/Liam_TwitterApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tweepy import API
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream


import json
import sys
import pandas as pd

In [0]:
CONSUMER_KEY = ''
CONSUMER_SECRET = ''

ACCESS_TOKEN = ''
ACCESS_TOKEN_SECRET = ''

In [0]:
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = API(auth, wait_on_rate_limit=True,
          wait_on_rate_limit_notify=True)

In [0]:
api = API(auth, wait_on_rate_limit=True,
          wait_on_rate_limit_notify=True)

class Listener(StreamListener):
    def __init__(self):
        super(Listener,self).__init__()
        
        self.output_list = [ ]
    def on_status(self, status):
        #exclude retweet
        if (not status.retweeted) and ('RT @' not in status.text):
            self.output_list.append(status)
        else:
            pass
     

    def on_error(self, status_code):
        print(status_code)
        return False


In [0]:
#output = open('stream_output.txt', 'w')

l = Listener()


In [0]:
stream = Stream(auth=api.auth, listener=l)

try:
    
    print('Start streaming.')
    #stream.sample(languages=['en'])
    
    
    #tracking target  SF - LA -122.75,36.8,-121.75,37.8, -118,34,-117,33
    #37.8,-74,40,-73,41
    stream.filter(#locations = [-122.75,36.8,-121.75,37.8, -118,33,-117,34],
                  track = ['movie', 'new movie', 'netflix', 'hulu', 'youtube', 'hbo'],
                  #is_async=True,
                  languages=['en'])
     

except KeyboardInterrupt:
    print("Stopped.")
finally:
    print('Done.')
    stream.disconnect()

Start streaming.
Stopped.
Done.


In [0]:
type(l.output_list[1])

tweepy.models.Status

In [0]:
l.output_list[0]._json.keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'display_text_range', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms'])

In [0]:
len(l.output_list)

185198

In [0]:
#convert jon to dataframe

dict_ = {'screen_name': [], 'id': [], 'location': [], 'created_at': [], 'text': [], 'favorite_count': [],
        'description': [], 'profile_background_color': [], 'profile_link_color': [],
        'profile_sidebar_border_color': [], 'profile_sidebar_fill_color': [], 'profile_text_color': []}


for i in range(0,len(l.output_list)):
    dict_['screen_name'].append(l.output_list[i]._json['user']['screen_name'])
    dict_['id'].append(l.output_list[i]._json['user']['id'])
    dict_['location'].append(l.output_list[i]._json['user']['location'])
    dict_['description'].append(l.output_list[i]._json['user']['description'])
    dict_['profile_background_color'].append(l.output_list[i]._json['user']['profile_background_color'])
    dict_['profile_link_color'].append(l.output_list[i]._json['user']['profile_link_color'])
    dict_['profile_sidebar_border_color'].append(l.output_list[i]._json['user']['profile_sidebar_border_color'])
    dict_['profile_sidebar_fill_color'].append(l.output_list[i]._json['user']['profile_sidebar_fill_color'])
    dict_['profile_text_color'].append(l.output_list[i]._json['user']['profile_text_color'])
    
    
    dict_['created_at'].append(l.output_list[i]._json['created_at'])
    dict_['text'].append(l.output_list[i]._json['text'])
    dict_['favorite_count'].append(l.output_list[i]._json['favorite_count'])


In [0]:
df = pd.DataFrame(dict_)
df.sort_values(by='created_at', inplace=True, ascending=False)
df.head(5)

In [0]:
pd.set_option('display.max_colwidth', -1)
print(df['text'])


In [0]:
df.to_csv('tweets14.csv')